In [1]:
import json
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
from selenium.webdriver.common.by import By

In [2]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

In [3]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
my_url = "https://soundcloud.com/discover"

In [5]:
driver = webdriver.Chrome()
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
client_id = soup.find("div", {"id": "g_id_intermediate_iframe"}).get("data-src")
client_id = ((client_id.split('?')[1]).split('&')[0]).split('=')[1]

In [6]:
client_id

'FjnXkiGFvyaVIYtXadMm9pqIDawoxzUW'

In [7]:
set_link = []

In [10]:
driver = webdriver.Chrome()
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
link = soup.find_all("a", {
    "class": "sc-link-secondary sc-link-light playableTile__usernameHeading audibleTile__usernameHeading sc-truncate sc-text-h4"})

for item in link:
    user_url = "https://soundcloud.com" + item.get("href")
    if user_url not in set_link:
        set_link.append(user_url)

for user_url in set_link:
    if len(set_link) > 25:
        break;
    driver.get(user_url)
    wait(driver, 'profileHeaderInfo__userName')
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    user_name = soup.find("h2", {"class": "profileHeaderInfo__userName"})

    # Get followers
    follower_url = user_url + "/following"
    driver.get(follower_url)

    #Wait
    scroll(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    followers = soup.find_all("a", {"class": "userBadgeListItem__heading"})
    for follower in followers:
        if follower not in set_link:
            set_link.append("https://soundcloud.com" + follower.get("href"))

In [12]:
len(set_link)

38

In [13]:
def get_all_user_data(set_link):
    #Get user id in SoundCloud/Discover by get meta tag in HEAD html
    set_data_user = []
    for link_user in set_link:
        driver = webdriver.Chrome()
        driver.get(link_user)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        #Get user id & user name & user url
        id_user = soup.find("meta", property="twitter:app:url:googleplay")
        set_data_user.append(id_user["content"].strip("soundcloud://users:"))

    return set_data_user

In [14]:
set_user = get_all_user_data(set_link)

In [16]:
len(set_user)

38

In [17]:
#============ using API to get USER.csv =================
#Get set_user API
def get_user_by_API(client_id):
    set_data_user = []
    # Select first column of the dataframe as a series
    for item in set_user:
        one_data_user = []
        user_id = item
        api_url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}'
        # get request
        r = requests.get(api_url)
        # check whether it get the url
        # or if it got request, does it have a string "incomplete_results":true in the response it recieved?
        while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
            time.sleep(1)
            r = requests.get(api_url)
        #JSON processing
        json_str = json.loads(r.text)
        one_data_user.append(str(json_str['id']))
        one_data_user.append(str(json_str['username']))
        one_data_user.append(str(json_str['permalink_url']))
        set_data_user.append(one_data_user)

    return set_data_user

In [18]:
get_user_by_API(client_id)

[['293713798', 'eevee', 'https://soundcloud.com/user-692461400'],
 ['6983206', 'Mah', 'https://soundcloud.com/mzn665'],
 ['150205120', 'lolovvveee2', 'https://soundcloud.com/lolollov2'],
 ['17971787', 'Agamidae', 'https://soundcloud.com/agamidae'],
 ['193531220', 'MugaTunes', 'https://soundcloud.com/mugatunesofficial'],
 ['2182746', 'Mighty', 'https://soundcloud.com/mightyj-1'],
 ['214279944', 'Ilyana Azman', 'https://soundcloud.com/ilyanaazman'],
 ['283347384', 'soo', 'https://soundcloud.com/yourparadis'],
 ['81561939', 'J e z i k a ♡', 'https://soundcloud.com/its-jezika-bruh'],
 ['2448383', 'Josh Kunze', 'https://soundcloud.com/kunze50'],
 ['119544823',
  'Khushpreet Singh',
  'https://soundcloud.com/khushpreet-singh-6'],
 ['454283658', 'Playlists Daily', 'https://soundcloud.com/partyplaylists69'],
 ['133467077',
  'Westcoast Radio (EDM)',
  'https://soundcloud.com/digitalstreams'],
 ['79816260', 'Austin24heck', 'https://soundcloud.com/austin24heck'],
 ['196864989', 'Billie Kihega', 

In [20]:
def getTracksAPI(set_user, client_id, limit=100):
    set_tracks = set()
    temp1 = 'https://api-v2.soundcloud.com/users/'
    temp2 = f'/tracks?client_id={client_id}&limit={limit}'

    for item in set_user:
        string_tracks = ''
        api = temp1 + item + temp2
        while True:
            r = requests.get(api)
            if r.status_code != 200:
                time.sleep(5.0)
            else:
                break
        for temp in r.json()['collection']:
            string_tracks = string_tracks + str(temp['id']) + ','
        tup = (item, string_tracks)
        set_tracks.add(tup)
    return set_tracks

In [21]:
getTracksAPI(set_user, client_id)

{('1', ''),
 ('1',
  '1035426364,1019999464,1017017875,994037122,911828809,907898896,903691249,831255739,825963814,623749281,612714387,611845218,594769962,483414039,471282132,363580073,353506475,347813702,344333772,332746526,332158279,319953335,316645648,315476469,313942963,313325317,310577244,305991908,304832401,304703680,303763719,298254464,297132472,295918712,271283998,253335407,225595558,222430653,220314818,218366427,216779504,214226516,191948995,191185189,190597571,169471267,'),
 ('1',
  '1060279630,1060274989,1060270390,237010350,235069910,235069919,235069927,235069931,233458601,141514159,70921906,70914182,70913911,70913715,70913159,35772577,'),
 ('1', '1113783223,1113796495,1113792481,'),
 ('1',
  '1138956514,1031540554,1026342412,989174902,966893845,938872219,921603301,914882944,891803494,866038042,856234912,843898141,805022032,796934149,788513644,783214762,776317807,768309526,761739898,756369826,746112265,736573639,726944263,703644604,698532229,643627437,616708656,615654291,59

In [22]:
def getPlaylistsAPI(user_id, client_id, limit=100):
    playlists = []
    url = f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={client_id}&limit={limit}'
    r = requests.get(url)

    while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
        time.sleep(1)
        r = requests.get(url)

    y = json.loads(r.text)
    if len(y['collection']) == 0:
        playlists.append([user_id, 'None', 'None'])
    else:
        for playlist in y['collection']:
            trackIDs = []
            for track in playlist['tracks']:
                trackIDs.append(str(track['id']))
            allTrackIDs = ','.join(trackIDs)
            playlists.append([user_id, str(playlist['id']), allTrackIDs])

    return playlists

In [23]:
playlistsAPI = [getPlaylistsAPI(x, client_id) for x in set_user]

wait
wait
wait
wait
wait
wait
wait
wait
wait
wait
wait
wait
wait
wait


KeyboardInterrupt: 